# Problem Set 6, Biomechanical Data
## Part A. Countermovement Jump (CMJ)

Using the force plate output from CMJs taken at 4 time periods during a season: 

1.	Calculate the jump height using the Impulse-Momentum Theorem for each trial. 

2.	Average the 3 trials for each testing session. 

3.	Plot the average jump heights for the 4 sessions. 

In [ ]:
# Helper Code

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Get the current working directory
current_directory = os.getcwd()

# Set path to the "Force_Plate_Data" folder
folder_path = os.path.join(current_directory, "Force_Plate_Data")


# Create an empty dictionary to store the DataFrames
dataframes_dict = {}

# Loop through the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Extract the file name without the extension
        name_without_extension = os.path.splitext(filename)[0]

        # Construct the full file path
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Store the DataFrame in the dictionary with the file name as the key
        dataframes_dict[name_without_extension] = df

'''      
After running this code, dataframes_dict will contain individual 
DataFrames for each CSV file in the folder. The keys of the 
dictionary will be the file names (without the extension), and 
the corresponding values will be the DataFrames.

For example, if you have files named "CMJ_1_1.csv" and "CMJ_1_2.csv" 
in the folder, the dictionary dataframes_dict will have keys "CMJ_1_1" 
and "CMJ_1_2," with each key having the corresponding DataFrame from 
the respective CSV file. You can access each DataFrame using its corresponding key:
'''
# END Helper Code
# -----------------------------------

# Function to Calculate Jump Height using the Impulse-Momentum Theorm 

#### 1. Calculate body weight from 1 second of quiet standing
#### 2. Find when force curve exceeds 5 standard deviations (SD) of body weight
#### 3. Set onset motion of CMJ as 30 ms before 5 SD threshold is breached
#### 4. Set off plate motion when force data = 0
#### 5. Caluclate impulse of force curve between onset and off plate time
#### 6. Calculate impulse of body weight between onset and off plate time
#### 7. Calculate net impulse
            Net impulse = force impulse - body weight impulse
#### 8. Calculate take-off velocity from the following equation
            Net Impulse = mass (in Kg) * velocity
#### 9. Calculate jump height using take-off velocity using the following equation
            Jump Height = 1/2 * (velocity^2 / gravity)


In [ ]:
# Check data loaded correctly: df_file1 should be 5612 rows 14 columns
df_file1

In [ ]:
# Plot a trial 
df_file1.plot(x=df_file1.columns[0], y=df_file1.columns[3])
plt.xlabel("Time (s)")
plt.ylabel("Combined GRF (N)")
plt.title("Countermovement Jump Force Output")
plt.show()

In [ ]:
#creating a function to calculate the jump height for each trial
gravity = 9.81 
SAMPLING_FREQUENCY = 1000

def calc_jump_height(dataframe):
    # Extract columns of interest - force and time
    force_data = df.iloc[:,3].values
    time_data = df.iloc[:,0]
    

    # Find the body weight (N) from the first second of data
    body_weight = np.mean(force_data[:SAMPLING_FREQUENCY])
    threshold_sd = 5 * np.std(force_data[:SAMPLING_FREQUENCY]) #Threshold 5*SD of bodyweight
    
    #Convert body mass from N to kg
    body_mass_kg = body_weight/gravity

    
    # Identify onset time
    start_n = body_weight - threshold_sd #below 5 SD
    start_p = body_weight + threshold_sd #above 5 SD
    
    #Initialize variables to track loop progress
    start_index = SAMPLING_FREQUENCY
    found_index = None
    # Start the loop after 1 second of data
    while start_index < len(force_data):
        if force_data[start_index] > start_p: # if force curve is POSITIVE @ onset movement
            found_index = start_index
            break
        elif force_data[start_index] < start_n: # if force curve is NEGATIVE @ onset movement
            found_index = start_index
            break
        start_index += 1
    # Calculate onset time 30 ms before curve is 5 SD above/below body weight
    onset_time = found_index - int(0.03 * SAMPLING_FREQUENCY)
    
    # Find off plate time and impulse time
    offplate_time = np.argmax(force_data == 0)
    impulse_time = (offplate_time - onset_time)/SAMPLING_FREQUENCY
    
    # Calculate impulse as area under the force curve btwn onset and offplate
    impulse_trap = np.trapz(force_data[onset_time:offplate_time], dx=1/SAMPLING_FREQUENCY)
    
    # Calculate impulse due to body weight (N)
    impulse_body_weight = body_weight * impulse_time
    
    # Calculate net impulse (which is normalized to body weight)
    net_impulse = impulse_trap - impulse_body_weight

    # Calculate velocity from impulse-momentum equation
    velocity = net_impulse/(body_mass_kg)

    #Calculate jump height
    jump_height = 0.5 * (velocity ** 2 / gravity)
    
    return jump_height

# Create a dictionary to store the jump height values for each jump
jump_heights = {}

#Calculate jump heights for each jump in the dictionary
for key, df in dataframes_dict.items():
    jump_height= calc_jump_height(df)
    jump_heights[key] = jump_height
    
#Print the calculated jump heights to verify
for key, height in jump_heights.items():
    print(f"Jump '{key}': Height = {height:.2f} meters")  

## Calculate Average Jump Height per Session

In [ ]:
# Create a dictonary to store jump heights for each session
session_jump_height = {}

# Aggregate and calculate jump heights by each session (4 sessions)
for key, df in dataframes_dict.items():
    session_name = "_".join(key.split('_')[:2]) # Here we want to extract the session name (e.g., 'CMJ_1')
    jump_height = calc_jump_height(df)
    
    if session_name not in session_jump_height:
        session_jump_height[session_name] = [jump_height]
    else:
        session_jump_height[session_name].append(jump_height)

# Calculate and print the average jump heights for each session
for session, heights in session_jump_height.items():
    print(f"Session {session}:")
    for jump_num, height in enumerate(heights, start=1):
        jump_id = f'{session}_{jump_num}'
        print(f"Jump '{jump_id}': Height = {height:.2f} meters")
    avg_height = np.mean(heights)
    print(f"Average Jump Height for Session {session}: {avg_height:.2f} meters")
    print()

## Plot Average Jump Heights

In [ ]:
# Extract session names and average jump heights for each
sessions = list(session_jump_height.keys())
avg_height = [np.mean(heights) for heights in session_jump_height.values()]

# Plot on a bar graph
plt.bar(sessions, avg_height, color = 'b')
plt.title('Average Jump Heights Across A Session')
plt.xlabel('Training Session')
plt.ylabel('Average Jump Height (m)')
plt.ylim(0,0.5)

# Get values for avg jump height at the top of each bar
for i in range(len(sessions)):
    plt.text(sessions[i], avg_height[i], f'{avg_height[i]:.2f}', ha='center', va='bottom')
plt.tight_layout()
plt.show


## Trouble Shooting

#### Code for 1 Trial

In [ ]:
# Test Code for one trial

# Load force plate data
df_file1 = dataframes_dict["CMJ_1_1"] 

# Set the global sampling frequency (in Hz)
SAMPLING_FREQUENCY = 1000

# Extract time and force columns
time = df_file1.iloc[:, 0].values
force_data = df_file1.iloc[:, 3].values

# Calculate body weight (N) as the average of the first second of data
body_weight = np.mean(force_data[:SAMPLING_FREQUENCY]) 
    
# Body mass in kg
body_mass_kg = body_weight/(9.81)

# Identify onset and off plate times
onset_threshold = body_weight - 5 * np.std(force_data[:SAMPLING_FREQUENCY])
onset_time = np.argmax(force_data < onset_threshold)
off_plate_time = np.argmax(force_data == 0)
impulse_time = (off_plate_time - onset_time)/SAMPLING_FREQUENCY

# Calculate impulse as the area under the force curve between onset and off plate times
impulse_trap = np.trapz(force_data[onset_time:off_plate_time], dx=1/SAMPLING_FREQUENCY)

# Calculate impulse (same as Trap method)
# impulse_Ns = force_data[onset_time:off_plate_time]*(1/SAMPLING_FREQUENCY)
# total_impulse = np.sum(impulse_Ns)

# Calculate impulse due to body weight (N)
impulse_body_weight = body_weight * impulse_time

# Net impulse (total impulse normalized to body weight)
net_impulse = impulse_trap - impulse_body_weight

# Impulse = m(in Kg) * velocity
takeoff_velocity = net_impulse / body_mass_kg

# Calculate jump height using velocity: Jump Height = 1/2 * (velocity^2 / gravity)
jump_height = 0.5 * (takeoff_velocity**2 / 9.81)


# Plot the force curve with onset and off plate lines
plt.plot(time, force_data, label='Force')
plt.axvline(x=df_file1.iloc[onset_time, 0], color='r', linestyle='--', label='Onset Time')
plt.axvline(x=df_file1.iloc[off_plate_time, 0], color='g', linestyle='--', label='Off Plate Time')
plt.xlabel("Time (s)")
plt.ylabel("Force (N)")
plt.title("Countermovement Jump Force Output")
plt.legend()
plt.grid(True)


# Display calculated jump height on the plot
plt.text(0.5, 0.5, f'Jump Height: {jump_height:.2f} meters', transform=plt.gca().transAxes)

plt.show()

# Print calculated jump height
print(f"Jump Height: {jump_height:.2f} meters")
# Print calculated take off velocity
print(f"Take off Velocity: {takeoff_velocity:.2f} m/s")

### Code for one trial (30 ms before threshold above/below body weight)

In [ ]:
# Test Code for one trial (30 ms before threshold above/below body weight)

# Load force plate data
df_file1 = dataframes_dict["CMJ_1_1"] 

# Set the global sampling frequency (in Hz)
SAMPLING_FREQUENCY = 1000

# Extract time and force columns
time = df_file1.iloc[:, 0].values
force_data = df_file1.iloc[:, 3].values

# Calculate body weight (N) as the average of the first second of data
body_weight = np.mean(force_data[:SAMPLING_FREQUENCY]) 
threshold_sd = 5 * np.std(force_data[:SAMPLING_FREQUENCY])
    
# Body mass in kg
body_mass_kg = body_weight/(9.81)

# Identify onset time
onset_threshold_n = body_weight - threshold_sd # below 5 sd
onset_threshold_p = body_weight + threshold_sd # above 5 sd

# Initialize variables to track loop progress
start_index = SAMPLING_FREQUENCY
found_index = None

# Start the loop after 1 second
while start_index < len(force_data):
    if force_data[start_index] > onset_threshold_p:
        found_index = start_index
        break
    elif force_data[start_index] < onset_threshold_n:
        found_index = start_index
        break
    start_index += 1

# Calculate onset time 30 ms before the found index
onset_time = found_index - int(0.03 * SAMPLING_FREQUENCY)

# Identify off plate time and impulse time
off_plate_time = np.argmax(force_data == 0)
impulse_time = (off_plate_time - onset_time)/SAMPLING_FREQUENCY

# Calculate impulse as the area under the force curve between onset and off plate times
impulse_trap = np.trapz(force_data[onset_time:off_plate_time], dx=1/SAMPLING_FREQUENCY)

# Calculate impulse due to body weight (N)
impulse_body_weight = body_weight * impulse_time

# Net impulse (total impulse normalized to body weight)
net_impulse = impulse_trap - impulse_body_weight

# Impulse = m(in Kg) * velocity
takeoff_velocity = net_impulse / body_mass_kg

# Calculate jump height using velocity: Jump Height = 1/2 * (velocity^2 / gravity)
jump_height = 0.5 * (takeoff_velocity**2 / 9.81)


# Plot the force curve with onset and off plate lines
plt.plot(time, force_data, label='Force')
plt.axvline(x=df_file1.iloc[onset_time, 0], color='r', linestyle='--', label='Onset Time')
plt.axvline(x=df_file1.iloc[off_plate_time, 0], color='g', linestyle='--', label='Off Plate Time')
plt.xlabel("Time (s)")
plt.ylabel("Force (N)")
plt.title("Countermovement Jump Force Output")
plt.legend()
plt.grid(True)


# Display calculated jump height on the plot
plt.text(0.6, 0.5, f'Jump Height: {jump_height:.2f} meters', transform=plt.gca().transAxes)

plt.show()

# Print calculated jump height
print(f"Jump Height: {jump_height:.2f} meters")
# Print calculated take off velocity
print(f"Take off Velocity: {takeoff_velocity:.2f} m/s")